In [10]:
# import libraries
import sys
import os
import pandas as pd
import matplotlib.pylab as plt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.utils import get_file_size
from compositionspace.io import get_reconstructed_positions, get_iontypes, get_iontypes_info
from compositionspace.preparation import ProcessPreparation
from compositionspace.segmentation import ProcessSegmentation
from compositionspace.clustering import ProcessClustering

In [2]:
# ! pip list  
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

Executing compositionspace in the following working directory: /home/kaiobach/Research/hu_hu_hu/sprint20/iuc09_saksena_comp_space/CompositionSpace


## Load reconstruction and ranging

In [3]:
workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/iuc09_saksena"
RECONSTRUCTION_AND_RANGING = (f"{workdir}/..")
RECONSTRUCTION_AND_RANGING = (f"{workdir}/PARAPROBE.Transcoder.Results.SimID.1.nxs",
                              f"{workdir}/PARAPROBE.Ranger.Results.SimID.1.nxs")
config_file_path = f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml"
results_file_path = f"{MY_PROCESSED_DATA_PATH}/CompositionSpace.Results.nxs"

In [4]:
# H5Web(RECONSTRUCTION_AND_RANGING[0])
# H5Web(RECONSTRUCTION_AND_RANGING[1])

In [5]:
(xyz_val, xyz_unit) = get_reconstructed_positions(RECONSTRUCTION_AND_RANGING[0])
ityp_info = get_iontypes_info(RECONSTRUCTION_AND_RANGING[0])
(ityp_val, ityp_unit) = get_iontypes(RECONSTRUCTION_AND_RANGING[1])

Load reconstructed positions shape (69507210, 3), type <class 'numpy.ndarray'>, dtype float32
Load iontypes information 111 types distinguished
Load ranged iontypes shape (69507210,), type <class 'numpy.ndarray'>, dtype uint8


## Voxelize with rectangular transfer function without creating slices

In [6]:
voxelize = ProcessPreparation(
    config_file_path,
    results_file_path,
    verbose=True)
voxelize.write_init_results()
voxelize.define_voxelization_grid(xyz_val)
voxelize.define_lookup_table(ityp_val)
voxelize.write_voxelization_grid_info()
voxelize.write_voxelization_results(ityp_info)

[[ 3.4028235e+38 -3.4028235e+38]
 [ 3.4028235e+38 -3.4028235e+38]
 [ 3.4028235e+38 -3.4028235e+38]]
shape (69507210,)
self.aabb3d [-55.  53.], extent 108
[-54. -53. -52. -51. -50. -49. -48. -47. -46. -45. -44. -43. -42. -41.
 -40. -39. -38. -37. -36. -35. -34. -33. -32. -31. -30. -29. -28. -27.
 -26. -25. -24. -23. -22. -21. -20. -19. -18. -17. -16. -15. -14. -13.
 -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.   0.   1.
   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.
  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.]
self.aabb3d [-59.  54.], extent 113
[-58. -57. -56. -55. -54. -53. -52. -51. -50. -49. -48. -47. -46. -45.
 -44. -43. -42. -41. -40. -39. -38. -37. -36. -35. -34. -33. -32. -31.
 -30. -29. -28. -27. -26. -25. -24. -23. -22. -21. -20. -19. -18. -17.
 -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -

In [7]:
get_file_size(results_file_path)
# H5Web(results_file_path)

216.965 MiB


<div class="alert alert-block alert-danger">
Discussion points:<br>
* Atomic decomposition<br>
</div>

## Segmentation PCA and IC minimization

In [8]:
segmentation = ProcessSegmentation(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
segmentation.perform_pca_and_write_results()
segmentation.perform_bics_minimization_and_write_results()

GaussianMixture ML analysis with n_cluster 1
GaussianMixture ML analysis with n_cluster 2
GaussianMixture ML analysis with n_cluster 3
GaussianMixture ML analysis with n_cluster 4
GaussianMixture ML analysis with n_cluster 5


In [10]:
get_file_size(results_file_path)
H5Web(results_file_path)

219.178 MiB


<jupyterlab_h5web.widget.H5Web object>

<div class="alert alert-block alert-danger">
Discussion points:<br>
* Modify NXapm_composition_space<br>
* What to show how to show?<br>
* Number of cluster vs Number of clusters? wording...?<br>
* Why to run the gm several times, ones in the bics loop ones in get composition cluster files 
</div>

## DBScan clustering

In [11]:
clustering = ProcessClustering(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
clustering.run_and_write_results()

n_ic_cluster 5, eps 3 nm, min_samples 5
np.shape(all_vxl_pos) (2489616, 3) list(set(phase_identifier) [0, 1, 2, 3, 4]
Loop 0
np.shape(trg_vxl_pos) (1108109, 3)
1
type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
Loop 1
np.shape(trg_vxl_pos) (922088, 3)
1
type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
Loop 2
np.shape(trg_vxl_pos) (119525, 3)
3
type(db.labels_) <class 'numpy.ndarray'> dtype int64
[-1  0  1]
Loop 3
np.shape(trg_vxl_pos) (339476, 3)
2
type(db.labels_) <class 'numpy.ndarray'> dtype int64
[-1  0]
Loop 4
np.shape(trg_vxl_pos) (418, 3)
1
type(db.labels_) <class 'numpy.ndarray'> dtype int64
[-1]


In [12]:
get_file_size(results_file_path)
H5Web(results_file_path)

219.283 MiB


<jupyterlab_h5web.widget.H5Web object>

<div class="alert alert-block alert-danger">
Discussion points:<br>
- Tests are too specific, hardcoded file names<br>
- Readthedocs documentation needs to be updated<br>
- GM and ML models are variables collect over<br>
- Loading file formats from the community should use ifes-apt-tc-data-modeling library currently using paraprobe result<br>
- Ion handling should use ifes-apt-tc-data-modeling is not added as a dependencies and loading properly<br>
-  tests/experiment_params.json should be removed?<br>
- NeXus renaming<br>
- CompositionSpace by design does not distinguish charge states iontypes should be atomic decomposed<br>
- Why is the center of the voxel defined by the median position of the ions but not by the barycenter of the voxel (currently using voxel barycenter)<br>
- Ran 2, and even 0.5 discretization speed is comparable<br>
- Triple loop in preparation step should be replaced with more fancy numpy indexing code that I know is somewhere but I couldnt find quickly<br>
- Move test data out of this repository<br>
</div>

## Meshing

Test for now with the SiGe dataset.

In [13]:
sige_file_path = "Output_DBSCAN_segmentation_phase_1.h5"
H5Web(sige_file_path)

<jupyterlab_h5web.widget.H5Web object>

In [5]:
import h5py
import numpy as np
with h5py.File(sige_file_path, "r") as h5r:
    n_vxls = 0
    aabb3d = np.zeros((3, 2), np.float64)
    for dim in [0, 1, 2]:
        aabb3d[dim, :] = [np.finfo(np.float64).max, np.finfo(np.float64).min]
    # print(aabb3d)
    for key in h5r["1"].keys():
        for dim in [0, 1, 2]:
            mimx = (np.min(h5r["1"][key][:, dim]), np.max(h5r["1"][key][:, dim]))
            if mimx[0] <= aabb3d[dim, 0]:
                aabb3d[dim, 0] = mimx[0]
            if mimx[1] >= aabb3d[dim, 1]:
                aabb3d[dim, 1] = mimx[1]
            n_vxls += int(np.shape(h5r["1"][key])[0])
    print(aabb3d)
    print(n_vxls)
    # assume cubic vxl 2nm edge length

[[ -94.18623883   87.23963464]
 [ -94.32663361   93.68342216]
 [-219.77029586   -0.74721716]]
590121


Assume that the data were discretized on the following rectangular grid with 2nm cubic voxel

In [7]:
nx = int((88--96)/2)
ny = int((94--96)/2)
nz = int((0--222)/2)
grid = np.zeros((nx, ny, nz), np.uint32)
# that grid should intentionally be a cuboid to enable checking correct dimensions

def i_to_xyz(i):
    z = int(i / (nx * ny))
    rem = i - (nx * ny * z)
    y = int(rem / nx)
    x = rem - (y * nx)
    return (x, y, z)

with h5py.File(sige_file_path, "r") as h5r:
    for key in h5r["1"].keys():
        jds = np.asarray(h5r["1"][key][:, 3], np.uint32)
        for j in jds:
            x, y, z = i_to_xyz(j)
            grid[x, y, z] = int(key) + 1
        print(key)
print(np.shape(grid))
print(np.unique(grid))

0
1
2
3
4
5
6
7
(92, 95, 111)
[0 1 2 3 4 5 6 7 8]


In [11]:
import h5py
with h5py.File("input.grid.nxs", "w") as h5w:
    h5w.create_dataset("/grid", compression="gzip", compression_opts=1, data=grid)

In [12]:
H5Web("input.grid.nxs")

<jupyterlab_h5web.widget.H5Web object>